In [11]:
!pip install langchain langchain-huggingface langchain_community faiss-cpu docx2txt -q

In [13]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
model_name = "HooshvareLab/bert-base-parsbert-uncased"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf_embedding = HuggingFaceEmbeddings(
model_name=model_name,
model_kwargs=model_kwargs,
encode_kwargs=encode_kwargs)

In [14]:
text = "افراد و آگاهی، همه دانش صحیح انتقال با هوشواره معتقدیم در ما"
embed = hf_embedding.embed_query(text)
print(f"طول embedding: {len(embed)}")

طول embedding: 768


In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [33]:
files = [
    {"path": "/content/esfahan.docx", "source": "qhazvin"},
    {"path": "/content/qhazvin.docx", "source": "tehran"},
    {"path": "/content/shiraz.docx", "source": "shiraz"}
]

all_chunks = []

for file in files:

  loader = Docx2txtLoader(file["path"])
  documents = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=500,
      chunk_overlap=50
  )
  chunks = text_splitter.split_documents(documents)

  for chunk in chunks:
      chunk.metadata["source"] = file["source"]
      all_chunks.append(chunk)

  print(f"فایل {file['path']} با موفقیت لود شد. تعداد تکه‌ها: {len(chunks)}")



فایل /content/esfahan.docx با موفقیت لود شد. تعداد تکه‌ها: 25
فایل /content/qhazvin.docx با موفقیت لود شد. تعداد تکه‌ها: 33
فایل /content/shiraz.docx با موفقیت لود شد. تعداد تکه‌ها: 24


In [21]:
from langchain_community.vectorstores import FAISS
import faiss

In [35]:
index = faiss.IndexFlatL2(len(hf_embedding.embed_query("سلام")))
vector_store = FAISS.from_documents(
    documents=all_chunks,
    embedding=hf_embedding
)

query = "غذاهای سنتی چیست؟"
results = vector_store.similarity_search(
    query,
    k=2,
    filter={"source": "tehran"}
)

print("\nنتایج جستجوی معنایی با فیلتر (منبع: tehran):")
for i, res in enumerate(results, 1):
    print(f"* نتیجه {i}: {res.page_content} [{res.metadata}]")


نتایج جستجوی معنایی با فیلتر (منبع: tehran):
* نتیجه 1: سفال و سرامیک

محصولات سفال و سرامیک نقش بسیار مهمی در صنایع‌دستی قزوین دارند. آثار سفالینه و سرامیکی با طرح‌ها و نقوش متنوع از جمله سوغاتی‌های محبوب این شهر هستند.

شیرینی‌های محلی

شیرینی‌های محلی مانند شیرینی قندی، پادرازی، نان چای و نان‌بادامی قزوین از جمله سوغاتی‌های خوشمزه و محبوب قزوین هستند.

عطر و ادکلن [{'source': 'tehran'}]
* نتیجه 2: شیرین‌پلو

شیرین پلو یکی از غذاهای سنتی و محبوب استان قزوین است که با طعم شیرین و دلچسب خود، طرفداران زیادی دارد. این غذا از ترکیب برنج، خلال پرتقال، شربت، خلال بادام، خلال پسته، زعفران، کره، سیب زمینی و سینه مرغ تهیه می‌شود.

 آش دوغ قزوین [{'source': 'tehran'}]
